## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [3]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        ####################
        # Declare model parameters
        ####################
        self.W1 = tf.Variable(shape=[28*28, 100], dtype=tf.float32,initial_value=tf.random.uniform(shape=[28*28, 100],minval=-0.1, maxval=0.1))

        self.b1 = tf.Variable(shape=[100], dtype=tf.float32, initial_value=tf.zeros(100))

        self.W2 = tf.Variable(shape=[100, 10], dtype=tf.float32,initial_value=tf.random.uniform(shape=[100, 10],minval=-0.1, maxval=0.1))

        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(10))
        self.trainable_variables = [self.W1, self.W2, self.b1, self.b2]

    def __call__(self, x):
        ####################
        # Implement model function
        ####################
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        h1 = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits

model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [4]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [6]:
train_data, test_data = mnist_dataset()
for epoch in range(200):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 1.87107 ; accuracy 0.5858667
epoch 1 : loss 1.8670747 ; accuracy 0.58778334
epoch 2 : loss 1.8630856 ; accuracy 0.58986664
epoch 3 : loss 1.8591031 ; accuracy 0.5915
epoch 4 : loss 1.8551271 ; accuracy 0.5935
epoch 5 : loss 1.8511574 ; accuracy 0.59501666
epoch 6 : loss 1.847194 ; accuracy 0.59683335
epoch 7 : loss 1.8432373 ; accuracy 0.59856665
epoch 8 : loss 1.8392868 ; accuracy 0.60003334
epoch 9 : loss 1.8353428 ; accuracy 0.60186666
epoch 10 : loss 1.8314052 ; accuracy 0.6036
epoch 11 : loss 1.8274742 ; accuracy 0.60515
epoch 12 : loss 1.8235496 ; accuracy 0.60681665
epoch 13 : loss 1.8196315 ; accuracy 0.60835
epoch 14 : loss 1.8157198 ; accuracy 0.60983336
epoch 15 : loss 1.8118148 ; accuracy 0.61115
epoch 16 : loss 1.8079162 ; accuracy 0.61265
epoch 17 : loss 1.8040241 ; accuracy 0.6142
epoch 18 : loss 1.8001386 ; accuracy 0.61551666
epoch 19 : loss 1.7962599 ; accuracy 0.61688334
epoch 20 : loss 1.7923875 ; accuracy 0.6185333
epoch 21 : loss 1.7885218 ; accurac